In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import scipy.io
import Rbeast as rb
from SLM_tools import *
from scipy.io import savemat
import pickle
import re
import time

In [ ]:
top_dir = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\Final Project\Project2-Omri and Idan\Results" 

pattern = re.compile(
    r"(?P<type>distance|energy)_vec_mu_(?P<mu>[-+]?\d*\.\d+|\d+)_energy_(?P<energy>[-+]?\d*\.\d+|\d+)_run_num_(?P<run_num>\d+)_total_num_target_\d+\.mat"
)
distance_paths = []
energy_paths = []
seen_distance_paths = set()
seen_energy_paths = set()
mu = 1.6
path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\testing data"
# save_path = os.path.join(path, rf"{mu}_v3")
# os.mkdir(save_path)
save_path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\testing data\merged_data_1_6"
for root, dirs, files in os.walk(top_dir):
    for file in files:
        match = pattern.match(file)
        if match:
            file_info = match.groupdict()
            if file_info['mu'] == str(mu):
                full_path = os.path.join(root, file)
                if file_info['type'] == 'distance':
                    if full_path not in seen_distance_paths:
                        distance_paths.append(full_path)
                        seen_distance_paths.add(full_path)
                elif file_info['type'] == 'energy':
                    if full_path not in seen_energy_paths:
                        energy_paths.append(full_path)
                        seen_energy_paths.add(full_path)


In [ ]:
C = []
downsampling_factor = 10000
n_components = 3
features_extracted = 0
total_runtime = 0
files_with_assembly = 0
total_data_files = len(energy_paths)
log = []
counters = {"load":[],"extraction":[],"segmentation":[],"pca":None,"extracted_features":[]}
for i, (energy_path, distance_path) in enumerate(zip(energy_paths, distance_paths)):
    start1 = time.time()
    distance_data = scipy.io.loadmat(distance_path)['foo']
    energy_data = scipy.io.loadmat(energy_path)['foo'].T
    energy_data, distance_data, time_vec = SLM_tools.downsample(energy_data, distance_data,downsampling_factor)
    merged = np.concatenate((energy_data, distance_data), axis=1)
    merged_filename = os.path.join(save_path, f"merged_energy_distance_{i}_mu_{mu}.mat")
    sio.savemat(merged_filename, {"energy_distance":merged})

In [ ]:
    loading_time = round(time.time() - start1,3)
    log1 = f"\nfile {i+1}/{len(energy_paths)}  loaded: {os.path.relpath(energy_path,top_dir)} Loading time: {loading_time} seconds"
    print(log1)
    log.append(log1)
    counters["load"].append(loading_time)
    start2 = time.time()
    o, cp, mean_trend = SLM_tools.beast(energy_data)
    segmentation_time = round(time.time() - start2,3)
    log2 = f"\nfinished segmentation(Beast), runtime: {segmentation_time} seconds"
    print(log2)
    log.append(log2)
    counters["segmentation"].append(segmentation_time)
    start3 = time.time()
    A = SLM_tools.feature_extraction(energy_data, distance_data,mean_trend,cp)
    extraction_time = round(time.time() - start3,3)
    print(f"\nfinished feature extraction, runtime: {extraction_time} seconds")
    counters["extraction"].append(extraction_time)
    print(f"\nAssembly? : {len(A) > 0}")
    if len(A) > 0:
        files_with_assembly += 1
        features_extracted += A[0].shape[0]
        counters["extracted_features"].append(A[0].shape[0])
        log3 = f"\nfiles_with_assembly: {files_with_assembly}/{total_data_files}"
        print(log3)
        log.append(log3)
        log4 = f"\nTotal features_extracted: {features_extracted}"
        print(log4)
        log.append(log4)
    else:
        log5 = f"\nNo Assembly, No features extracted"
        print(log5)
        log.append(log5)
    C.append(A)
    total_runtime += (loading_time + extraction_time + segmentation_time)
log6 = f"\nFinished pre-pca processing,total runtime {total_runtime} for {len(energy_paths)} files"
print(log6)
log.append(log6)
c_path = os.path.join(save_path,f"C_all_mu_{mu}.pkl")
with open(c_path, "wb") as f:
    pickle.dump(C, f)
log7 = "\nsaved all processed data"
print(log7)
log.append(log7)
start_pca = time.time()
log8 = f"\nStarting PCA"
print(log8)
log.append(log8)
c_reduced = SLM_tools.feature_selection(C)
principal_components, score, latent = SLM_tools.pca(c_reduced,n_components)
a_reduced = SLM_tools.data_preparation(score, c_reduced,n_components)
counters["selected_features"] = a_reduced.shape[0]
runtime_pca = round(time.time() - start_pca,3)
counters["pca"] = runtime_pca
total_runtime += runtime_pca
log9 = f"\nFinished PCA runtime {runtime_pca} seconds"
print(log9)
log.append(log9)
a_path = os.path.join(save_path,f"a_reduced_all_mu_{mu}.mat")
savemat(a_path, {'a_reduced': a_reduced})
log10 = f"\nsaved all extracted features , total extracted features {features_extracted} total runtime {total_runtime} seconds"
log.append(log10)
log.append("SUMMARY")
log11 = f"\nfiles {total_data_files} \nfiles_with_assembly: {files_with_assembly}/{total_data_files} \nmean extracted features {np.mean(counters['extracted_features'])} \ntotal extracted features {features_extracted} \ntotal_selected_features {counters['selected_features']} \ntotal runtime {total_runtime} seconds \nmean loading time {np.mean(counters['load'])} \nmean segmentation time {np.mean(counters['segmentation'])} \nmean extraction time {np.mean(counters['extraction'])} \nPCA time {counters['pca']}"
log.append(log11)
print(log11)
log_path = os.path.join(save_path,f"run_log_mu_{mu}.txt",)
with open(log_path,'w') as f:
    for line in log:
        f.write(line)
        

In [1]:
import os
import numpy as np
from scipy import interpolate
import scipy.io
import Rbeast as rb
from SLM_tools import *
from scipy.io import savemat
import pickle
import re
import time

In [2]:
a_reduced = sio.loadmat(r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI_TEMP\testing data\out\out_21_8\selected_features_21_08_20_41.mat")["selected_features"]

In [3]:
a_reduced.shape

(2615, 4)

In [4]:
path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\testing data"
save_path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI_TEMP\testing data\out\out_21_8\run_2"
os.makedirs(save_path,exist_ok=True)

In [5]:
YI, tfas_predict_mat, tfas_actually_mat, mean_error_mat,random_x = SLM_tools.model_training_with_cv(a_reduced,n_components=3,cv_num=10)

In [6]:
SLM_tools.draw_stochastic_landscape_2d(a_reduced,save_path,3)

In [7]:
cv_num = 10
mean_vec, std_vec, hist_space, x_hist_space, x_ticks, y_ticks = SLM_tools.model_eval(tfas_predict_mat, tfas_actually_mat, cv_num, save_path)

In [8]:
tfas_predict_mat_2, tfas_actually_mat_2, mean_error_mat_2 = SLM_tools.train_again_on_validation(a_reduced, n_components=3)

In [9]:
SLM_tools.cv_bias_correction(tfas_predict_mat_2=tfas_predict_mat_2, tfas_actually_mat_2=tfas_actually_mat_2, hist_space=hist_space, mean_vec=mean_vec, x_hist_space=x_hist_space,x_ticks=x_ticks,y_ticks=y_ticks,save_path=save_path)

In [ ]:
data = sio.loadmat(r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\data_for_box2.mat")["data"]
YY = sio.loadmat(r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\YY_for_box2.mat")["YY"]
fig, ax = plt.subplots()
flat_data = []
for sublist_group in data:
    for arr in sublist_group:
        if arr.ndim > 2:
            raise ValueError(f"Array with shape {arr.shape} is too high-dimensional for a boxplot.")
        elif arr.ndim == 2:
            # Flatten the array to 1D, or choose a specific axis, depending on the data
            arr = arr.flatten()
        flat_data.append(arr)
bp = ax.boxplot(flat_data, positions=np.round(YY.flatten(), 2), patch_artist=True)
for i, box in enumerate(bp['boxes']):
    box.set(facecolor='white', edgecolor='blue', linewidth=2)

# Set color for medians, whiskers, and caps
for i, element in enumerate(bp['medians'] + bp['whiskers'] + bp['caps']):
    element.set_color('black')

# Set color for fliers
for i, element in enumerate(bp['fliers']):
    element.set_markeredgecolor('black')
plt.show()

In [ ]:
%reset -f

In [ ]:
import os
import numpy as np
from scipy import interpolate
import scipy.io
import Rbeast as rb
from SLM_tools import *
from scipy.io import savemat
import pickle
import re
import time

In [ ]:
top_dir = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI_TEMP\testing data\irregular_data"

pattern = re.compile(
    r"mini_(?P<type>denial|summed_distance_UP|summed_energy_UP)_vec_mu_(?P<mu>[-+]?\d*\.\d+|\d+)_energy_(?P<energy>[-+]?\d*\.\d+|\d+)_run_num_(?P<run_num>\d+)_total_num_target_\d+\.mat"
)

denial_paths = []
summed_distance_UP_paths = []
summed_energy_UP_paths = []
seen_denial_paths = set()
seen_summed_distance_UP_paths = set()
seen_summed_energy_UP_paths = set()

mu = 3  # Set this to the desired mu value
energy = 3.5  # Set this to the desired energy value

save_path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI_TEMP\testing data\meged_data_deniel_energy_3_5_distance_m_3"
os.makedirs(save_path, exist_ok=True)

for root, dirs, files in os.walk(top_dir):
    for file in files:
        match = pattern.match(file)
        if match:
            file_info = match.groupdict()
            if file_info['energy'] == str(energy) and file_info['mu'] == str(mu) :
                full_path = os.path.join(root, file)
                if file_info['type'] == 'denial':
                    if full_path not in seen_denial_paths:
                        denial_paths.append(full_path)
                        seen_denial_paths.add(full_path)
                elif file_info['type'] == 'summed_distance_UP':
                    if full_path not in seen_summed_distance_UP_paths:
                        summed_distance_UP_paths.append(full_path)
                        seen_summed_distance_UP_paths.add(full_path)
                elif file_info['type'] == 'summed_energy_UP':
                    if full_path not in seen_summed_energy_UP_paths:
                        summed_energy_UP_paths.append(full_path)
                        seen_summed_energy_UP_paths.add(full_path)

In [ ]:
print(f"energy_len: {len(summed_energy_UP_paths)},distance_len: {len(summed_distance_UP_paths)}, denial_len: {len(denial_paths)}")

In [ ]:
zeros = 0
for i in range(len(denial_paths)):
    denial_data = np.cumsum(scipy.io.loadmat(denial_paths[i])['foo'][:,0])
    summed_energy_UP_data = np.squeeze(scipy.io.loadmat(summed_energy_UP_paths[i])['foo'])
    summed_distance_UP_data = scipy.io.loadmat(summed_distance_UP_paths[i])['foo']
    # values_vec, distance_data, time_vec = SLM_tools.interpolate_data_over_regular_time(summed_energy_UP_data, summed_distance_UP_data, denial_data)
    zeros +=np.count_nonzero(summed_distance_UP_data == 0)

In [ ]:
zeros

In [ ]:
zeros

In [ ]:
np.count_nonzero(distance_data == 0)

In [ ]:
# downsampling_factor = 1  # Set this to your desired downsampling factor
for i, (denial_path, summed_energy_UP_path, summed_distance_UP_path) in enumerate(zip(denial_paths, summed_energy_UP_paths, summed_distance_UP_paths)):
    start1 = time.time()
    
    denial_data = np.cumsum(scipy.io.loadmat(denial_path)['foo'][:,0])
    summed_energy_UP_data = np.squeeze(scipy.io.loadmat(summed_energy_UP_path)['foo'])
    summed_distance_UP_data = scipy.io.loadmat(summed_distance_UP_path)['foo']
    
    # Assuming we need to downsample all three datasets
    # denial_data, summed_energy_UP_data, time_vec = SLM_tools.downsample(denial_data, summed_energy_UP_data, downsampling_factor)
    # _, summed_distance_UP_data, _ = SLM_tools.downsample(denial_data, summed_distance_UP_data, downsampling_factor)
    
    merged = np.concatenate((np.reshape(denial_data,(len(denial_data),1)), np.reshape(summed_energy_UP_data,(len(summed_energy_UP_data),1)), summed_distance_UP_data), axis=1)
    
    merged_filename = os.path.join(save_path, f"merged_denial_energy_distance_{i}_mu_{mu}_energy_{2.2}.mat")
    sio.savemat(merged_filename, {"merged_data": merged})
    
    end1 = time.time()
    print(f"Processed and saved file {i+1}/{len(denial_paths)} in {end1 - start1:.2f} seconds")

print("All files processed and saved.")

In [ ]:
from SLM_tools import *
dir_path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI_TEMP\testing data\merged_data_1_6"
n_components = 3
downsampling_factor = None
target_num = 2
time_vec_exists = False
cv_num = 10 
save_path = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI_TEMP\testing data\out\out_2"
os.makedirs(save_path,exist_ok=True)
data_variable_name = "energy_distance"
SLM_tools.create_and_evaluate_stochastic_landscape(dir_path,n_components, downsampling_factor, target_num, time_vec_exists, cv_num, save_path, data_variable_name)

In [1]:
import cv2
img = cv2.imread("SLM_logo.png")
img = cv2.resize(img,(500,1024))
cv2.imshow("img",img)
cv2.waitKey(0)

error: OpenCV(4.6.0) C:\b\abs_f8n1j3l9l0\croot\opencv-suite_1691622637237\work\modules\highgui\src\window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
from SLM_tools import *
import scipy as sio
feature_matrix = sio.loadmat(r"C:\Users\Omri.k\Desktop\temp\testing data\testing data\out\out_28_8\29_8\selected_features_29_08_11_35.mat")["feature_matrix"]
save_path = r"C:\Users\Omri.k\Desktop\temp\testing data\testing data\out\out_28_8\29_8\tests"

In [ ]:
feature_matrix.shape

In [ ]:
YI, tfas_predict_mat, tfas_actually_mat, mean_error_mat, random_x = SLM_tools.model_training_with_cv(feature_matrix,
                                                                                                     3,
                                                                                                    int(10))


In [ ]:
mean_vec, std_vec, hist_space, x_hist_space, x_ticks, y_ticks = SLM_tools.model_eval(tfas_predict_mat,
                                                                                             tfas_actually_mat,
                                                                                             int(10),
                                                                                             save_path)

In [ ]:
tfas_predict_mat_2, tfas_actually_mat_2, mean_error_mat_2 = SLM_tools.train_again_on_validation(feature_matrix,
                                                                                                        3)
SLM_tools.cv_bias_correction(tfas_predict_mat_2, tfas_actually_mat_2, hist_space, mean_vec, x_hist_space,
                                     x_ticks, y_ticks, save_path)